# DOCUMENTATION: CORPUS CREATION

In [14]:
# For importing the data
import sqlite3
from pathlib import Path
import gzip

# For preprocessing
import re
import fasttext
from huggingface_hub import hf_hub_download
import spacy
import dateparser

# Other
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from collections import Counter

# 📥 Import Data

## 📥 Import Open Library into SQLite
The [Open Library](https://openlibrary.org/developers/dumps) provides monthly data dumps as tab separated text files containing:
- type - type of record (/type/edition, /type/work etc.)
- key - unique key of the record (/books/OL1M etc.)
- revision - revision number of the record
- last_modified - last modified timestamp
- JSON - the complete record in JSON format

This project uses the following files which need to be downloaded from the [Open Library Data Dump, 2025-06-30](https://archive.org/details/ol_dump_2025-06-30), before running this script (Newer dumps can be used but will yield slitely different results, change *ol_dump_date*):
- the works dump (~ 3.7GB, 39.831.591 items)
- and the authors dump (~ 0.7GB, 14.547.394 items).



The first code section is adapted from [skeptric.com (Importing Open Library into SQLite)](https://skeptric.com/openlibrary-sqlite), originally created by Edward Ross.  
Retrieved on July 27, 2025.

In [5]:
ol_dump_date = "2025-06-30"
data_path = Path("")

# Define the path to the dump files
def ol_path(segment):
    return data_path / f"ol_dump_{segment}_{ol_dump_date}.txt.gz"

# Iterate over the dump file to yield tuples corresponding to the rows in the SQLite table
def ol_data(segment):
    with gzip.open(ol_path(segment), "rt") as f:
        for line in f:
            yield tuple(line.split("\t", 5))

# Create minibatches from the tuples for efficiency
def minibatch(seq, size):
    items = []
    for x in seq:
        items.append(x)
        if len(items) >= size:
            yield items
            items = []
    if items:
        yield items

# Built tables
def create_segment(cur, segment, batch_size=10_000):
    cur.execute(f"DROP TABLE IF EXISTS {segment}")
    cur.execute(f"CREATE TABLE {segment} (type TEXT, key TEXT, revision INT, last_modified TEXT, json TEXT);")
    
    with con:
        for batch in minibatch(tqdm(ol_data(segment)), batch_size):
            con.executemany(f"INSERT INTO {segment} VALUES (?,?,?,?,?)", batch)

In [16]:
# Connect to SQLite database file
con = sqlite3.connect("openlibrary.sqlite")

con.execute("PRAGMA synchronous=OFF")
con.execute("PRAGMA count_changes=OFF")
con.execute("PRAGMA journal_mode=MEMORY")
con.execute("PRAGMA temp_store=MEMORY")

In [7]:
# Create "works" table and "authors" table in SQLite database
for segment in ["works", "authors"]:
    create_segment(con, segment)

39831591it [05:12, 127522.74it/s]
14547394it [01:12, 201391.26it/s]


## 🔍 Explore SQLite file

In [9]:
def inspect_json_schema(table, sample_size=10000):
    # Load a sample of JSON strings
    rows = con.execute(f"SELECT json FROM {table} LIMIT {sample_size}").fetchall()
    
    # Collect all top-level keys
    key_counter = Counter()
    for row in rows:
        try:
            data = json.loads(row[0])
            key_counter.update(data.keys())
        except Exception:
            continue  # skip invalid JSON rows
    
    # Convert to DataFrame for better display
    df_keys = pd.DataFrame(key_counter.items(), columns=["attribute", "count"])
    df_keys.sort_values(by="count", ascending=False, inplace=True)
    df_keys.reset_index(drop=True, inplace=True)
    
    return df_keys

In [10]:
# works table
pd.read_sql_query("PRAGMA table_info(works)", con)

,cid,name,type,notnull,dflt_value,pk
0,0,type,TEXT,0,None,0
1,1,key,TEXT,0,None,0
2,2,revision,INT,0,None,0
3,3,last_modified,TEXT,0,None,0
4,4,json,TEXT,0,None,0


In [11]:
# authors table
pd.read_sql_query("PRAGMA table_info(authors)", con)

,cid,name,type,notnull,dflt_value,pk
0,0,type,TEXT,0,None,0
1,1,key,TEXT,0,None,0
2,2,revision,INT,0,None,0
3,3,last_modified,TEXT,0,None,0
4,4,json,TEXT,0,None,0


In [12]:
# attributes in the works file
inspect_json_schema("works")

,attribute,count
0,title,10000
1,created,10000
2,last_modified,10000
3,latest_revision,10000
4,key,10000
5,authors,10000
6,type,10000
7,revision,10000
8,subjects,5641
9,subject_places,2579


There is no description for many works, though still enough for a sufficiently large data set.

In [14]:
# attributes in the authors file
inspect_json_schema("authors")

,attribute,count
0,type,10000
1,key,10000
2,revision,10000
3,last_modified,10000
4,name,9984
5,latest_revision,9122
6,created,9120
7,source_records,8996
8,personal_name,3059
9,birth_date,857


# 🛠️ Construct Final Dataset

The SQLite now contains many records that are not useful for this project as they do not have a description. 
Therefore, the works *with* description are now extracted into a DataFrame for further processing.

First, we retrieve the following information from the *works table*:
- title,
- authors keys,
- description,
- and first publication date (if available)

Descriptions that are neither a book summary not written in English are dropped from the DataFrame.

The *works table* only references authors by their keys 
e.g. "[{"type":"/type/author_role","author":{"key":"/authors/OL24953A"}}]".
So author names need to be pulled separately from the *authors table* via the key.

The publication dates are in verious different formats and need to be normalized. Only the year will be kept.

In [18]:
# Build Dataframe with all works that have a description

def get_works_with_description(db=con):
    query = """
    SELECT json_extract(json, '$.title') AS title, 
           json_extract(json, '$.authors') AS authors,
           json_extract(json, '$.first_publish_date') AS pub_date,
           COALESCE(
               json_extract(json, '$.description.value'),
               json_extract(json, '$.description')
           ) AS description
    FROM works
    WHERE (json_extract(json, '$.description.value') IS NOT NULL
           OR json_extract(json, '$.description') IS NOT NULL);
    """
    
    df = pd.read_sql_query(query, db)
    
    df["authors"] = df["authors"].apply(extract_author_keys)

    return df


# Filter descriptions 
def filter_descriptions(x):
    # regular expressions for html-tag, links etc.
    rules = [r"<.*>.*<\/.*>", 
         r"\d{2} cm",
         r"Available in .* format",
         r"https?:",
         r"www.",
         r"@"]
    pattern = "|".join(f"({r})" for r in rules)

    # remove descriptions that match the regex
    # return the rest but without newline-characters (necessary for language detection with fasttext)
    if bool(re.search(pattern, x)):
        return None
    else:
        x = x.replace("\n", " ")
        return x


# Extract language label and probability from fastText classification output
def lang_label(label_tupel):
    label = label_tupel[0][0]
    return label

def lang_prob(label_tupel):
    prob = label_tupel[1][0]
    return prob


# Extract author keys from json string
def extract_author_keys(authors_str):
    try:
        authors = json.loads(authors_str)
        return [a["author"]["key"] for a in authors if "author" in a and "key" in a["author"]]
    except:
        return []


# Extract author keys and names from authors dump as {key: name, ...}
def get_author_name_mapping(db=con):

    query = """
    SELECT 
        json_extract(json, '$.key') AS key,
        json_extract(json, '$.name') AS name
    FROM authors
    """
    
    df = pd.read_sql_query(query, db)
    
    return dict(zip(df["key"], df["name"]))


# Replace author keys with names
def replace_author_name(keys):
    return [author_map.get(k) for k in keys if k in author_map]


# Process date information, only keep publication year
def process_dates(x):
    if pd.isna(x):
        return None
    parsed = dateparser.parse(x)
    return parsed.year if parsed else None


## 🛠️ Build Dataframe with all works having a description

In [48]:
# Load relevant works
df_raw = get_works_with_description()
df_raw

,title,authors,pub_date,description
0,Modern masters of Kyoto,[/authors/OL24953A],None,"""In a series of intertwined narratives, Porcel..."
1,Daftar-i shir,[/authors/OL4089652A],None,Poems in Sorani dialect.
2,Kichocheo cha fasihi,[/authors/OL96301A],None,On the role and importance of oral literature ...
3,Şimdi haberler,[/authors/OL97075A],None,Poems.
4,"Kimse kızmasın, kendimi yazdım",[/authors/OL97128A],None,Journalism and journalists; Turkey; autobiogra...
...,...,...,...,...
1639565,The Witch (Tales of Montague Dragon),[/authors/OL3864482A],None,This is the third book in the *Tales of Montag...
1639566,Epidemic,[/authors/OL3866013A],None,"At the threshold of the third millennium, we a..."
1639567,Declination,[/authors/OL3869731A],None,A continuation of the saga begun in *Right Asc...
1639568,La politique des reliques de Constantin à Sain...,[/authors/OL3933873A],None,"Jusqu'a ce jour, les reliques du Christ, de la..."


## ⚙️ Filter descriptions

Not every description contains a summary of the book. Some only give information e.g. about the genre, the size or the source of the book or about assosiations linked to the book. 

In [51]:
# First, only keep works with a unique description:
df_unique = df_raw.drop_duplicates(subset=["description"], keep=False)

In [52]:
# Drop rows specific rows that contain html-tags, urls etc.
df_unique.loc[:,"description"] = df_unique["description"].progress_apply(filter_descriptions)
df_clean = df_unique[df_unique["description"].notna()].copy()
df_clean

100%|██████████████████████████████| 1469208/1469208 [01:04<00:00, 22766.75it/s]


,title,authors,pub_date,description
0,Modern masters of Kyoto,[/authors/OL24953A],None,"""In a series of intertwined narratives, Porcel..."
2,Kichocheo cha fasihi,[/authors/OL96301A],None,On the role and importance of oral literature ...
4,"Kimse kızmasın, kendimi yazdım",[/authors/OL97128A],None,Journalism and journalists; Turkey; autobiogra...
5,Klâsikler tartışması,[/authors/OL97249A],None,World literature; Turkey; history and criticism.
7,Az mafahim-i Qur'an,[/authors/OL4180247A],None,Commentary on various topics as narrated in th...
...,...,...,...,...
1639562,The Depression,[/authors/OL3800609A],None,Its a story about a woman in the depression li...
1639565,The Witch (Tales of Montague Dragon),[/authors/OL3864482A],None,This is the third book in the *Tales of Montag...
1639566,Epidemic,[/authors/OL3866013A],None,"At the threshold of the third millennium, we a..."
1639568,La politique des reliques de Constantin à Sain...,[/authors/OL3933873A],None,"Jusqu'a ce jour, les reliques du Christ, de la..."


### 💬 Drop all non-english descriptions

In [54]:
# Load fastText language identification Tmodel from huggingface
model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(model_path)

In [55]:
# Identify languages
df_clean["language"] = df_clean["description"].progress_apply(model.predict)

100%|███████████████████████████████| 1420027/1420027 [05:23<00:00, 4389.15it/s]


In [56]:
# Extract language label from fastText classification output
df_clean["language_prob"] = df_clean["language"].apply(lang_prob)
df_clean["language"] = df_clean["language"].apply(lang_label)

In [57]:
# Only keep rows with english descriptions (high probability)
df_english = df_clean[df_clean["language"]=="__label__eng_Latn"].copy()
df = df_english[df_english["language_prob"]>0.999].copy()

In [58]:
df = df.drop(columns=["language", "language_prob"])
df = df.reset_index(drop=True)
df

,title,authors,pub_date,description
0,Modern masters of Kyoto,[/authors/OL24953A],None,"""In a series of intertwined narratives, Porcel..."
1,Kichocheo cha fasihi,[/authors/OL96301A],None,On the role and importance of oral literature ...
2,Az mafahim-i Qur'an,[/authors/OL4180247A],None,Commentary on various topics as narrated in th...
3,Learning to Live with Diabetes,[/authors/OL4189170A],None,"As back as 1960, a Diabetes clinic was started..."
4,"Nư̄a, tai, ʻō̜k, tok",[/authors/OL97664A],None,Commentaries of Thai and foreign writers and l...
...,...,...,...,...
812767,A love to last forever,[/authors/OL18777A],None,All Beth Gallatin has really wanted out of lif...
812768,The Depression,[/authors/OL3800609A],None,Its a story about a woman in the depression li...
812769,The Witch (Tales of Montague Dragon),[/authors/OL3864482A],None,This is the third book in the *Tales of Montag...
812770,Epidemic,[/authors/OL3866013A],None,"At the threshold of the third millennium, we a..."


## ⚙️ Normalize publication dates

This approach was adapted from Chris Dixon on [Stackoverflow](https://stackoverflow.com/a/69819660).
Retrieved on July 29, 2025.

In [61]:
# Parse publication dates & get the year
df["pub_date"] = df["pub_date"].progress_apply(process_dates)
df

100%|██████████████████████████████████| 812772/812772 [16:41<00:00, 811.34it/s]


,title,authors,pub_date,description
0,Modern masters of Kyoto,[/authors/OL24953A],NaN,"""In a series of intertwined narratives, Porcel..."
1,Kichocheo cha fasihi,[/authors/OL96301A],NaN,On the role and importance of oral literature ...
2,Az mafahim-i Qur'an,[/authors/OL4180247A],NaN,Commentary on various topics as narrated in th...
3,Learning to Live with Diabetes,[/authors/OL4189170A],NaN,"As back as 1960, a Diabetes clinic was started..."
4,"Nư̄a, tai, ʻō̜k, tok",[/authors/OL97664A],NaN,Commentaries of Thai and foreign writers and l...
...,...,...,...,...
812767,A love to last forever,[/authors/OL18777A],NaN,All Beth Gallatin has really wanted out of lif...
812768,The Depression,[/authors/OL3800609A],NaN,Its a story about a woman in the depression li...
812769,The Witch (Tales of Montague Dragon),[/authors/OL3864482A],NaN,This is the third book in the *Tales of Montag...
812770,Epidemic,[/authors/OL3866013A],NaN,"At the threshold of the third millennium, we a..."


In [62]:
# Handling invalid dates
df.loc[df["pub_date"] > 2025, "pub_date"] = float("NaN")

In [63]:
# Remove books without publication date
df = df[df["pub_date"].notna()].copy()

## ⚙️ Enrich DataFrame with author names

In [65]:
# Load author mapping as {key: name, ...}
author_map = get_author_name_mapping()

In [66]:
# Replace author keys with names
df["authors"] = df["authors"].progress_apply(replace_author_name)
df

100%|███████████████████████████████| 140747/140747 [00:00<00:00, 242170.03it/s]


,title,authors,pub_date,description
62,The September sisters,[Jillian Cantor],2009.0,A teenaged girl tries to keep her family and h...
63,Lush,"[Sasha White, Sasha White]",2007.0,The pleasure is all yours if you’re ready to l...
64,Saṃvidhānasabhā ra rājyako rupāntaraṇa,[Khimalāla Devakoṭā],2007.0,Constituent assembly and federal restructuring...
65,Ḥikāyah kull khamīs,[ʻAbd Allāh ʻAbbās Iryānī],2006.0,Novel and play.
66,Nikog nema doma,[Dubravka Ugrešić],2005.0,"Taking us on travels through Europe, and acros..."
...,...,...,...,...
812017,The Tender Texan,[Jodi Thomas],1991.0,A Shocking Proposition\r \r Bold and beautiful...
812018,Platitudes in the making,[Holbrook Jackson],1911.0,"In 1955, in a used bookstore in San Francisco,..."
812255,The amateur astronomer,[Patrick Moore],1974.0,"Sir Patrick Moore, CBE, FRS is without a doubt..."
812455,"Our Nig, or, Sketches from the life of a free ...",[Harriet E. Wilson],1984.0,"""A fusion of two literary modes of the ninetee..."


# 🔍 Overview

In [21]:
# Stat summary of publication years
df["pub_date"].describe()

count    140829.000000
mean       1990.603185
std          23.385780
min        1001.000000
25%        1988.000000
50%        1996.000000
75%        2001.000000
max        2018.000000
Name: pub_date, dtype: float64

# 📤 Save DataFrame in JSON format

In [90]:
data = df.to_json("data.json", orient="records"

In [20]:
con.close()